<a href="https://colab.research.google.com/github/pedroteixeirazup/Keras/blob/master/BreastCancer/TrainBreastCancerKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D, Activation
from keras.models import Model
from keras import optimizers
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_dir     = '/content/drive/My Drive/Colab Notebooks/Data/train/'
test_dir      = '/content/drive/My Drive/Colab Notebooks/Data/test/'
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(255,255), batch_size=128,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir,target_size=(255,255), batch_size=128,class_mode='categorical')

Found 795 images belonging to 2 classes.
Found 275 images belonging to 2 classes.


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output

x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(2, activation=tf.nn.softmax)(x)

model = Model(inputs=base_model.input, output=predictions)

for layer in base_model.layers:
    layer.trainable = False

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [ ]:
adam = optimizers.adam(lr=0.0001)

model.compile(optimizer=adam, loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/Data/model.{epoch:02d}-{val_accuracy:.2f}.hdf5', monitor='val_accuracy',\
 verbose=1, save_best_only=True, mode='max', period=1)]
 
history = model.fit_generator(train_generator, verbose=1, steps_per_epoch=100, epochs=10, \
 validation_data=test_generator, validation_steps=275, callbacks=callbacks)

Epoch 1/10
100/100 [==============================] - 6398s 64s/step - loss: 0.2147 - accuracy: 0.9387 - val_loss: 1.5793 - val_accuracy: 0.6110

Epoch 00001: val_accuracy improved from -inf to 0.61097, saving model to /content/drive/My Drive/Colab Notebooks/Data/model.01-0.61.hdf5
Epoch 2/10
 99/100 [============================>.] - ETA: 19s - loss: 0.1623 - accuracy: 0.9561